# In this file data about groups, events and RSVPs are gathered, filtered and transferred to CSV files.

# 1. Preparation: required libraries are fetched and connection to meetup is established.

In [26]:
    !pip install meetup.api
    import meetup.api
    import json

    import pandas
    from pandas.io.json import json_normalize

    
    client = meetup.api.Client('149155b67979442634b463d11cc')

  Using cached https://files.pythonhosted.org/packages/b7/44/a545b860f19cac088cb7f3e39beae8334b63e025729348ddeffb3342122a/meetup_api-0.1.1-py2.py3-none-any.whl


# 2. function createCSV is prepared

In [27]:
    # creation of the CSV files for Groups, Events and RSVPs
    def createCSV(fileGroups, fileRSVPs ,fileEvents, eventsRange):
        groups =""
        f = open(fileRSVPs, "w+")
        f.close()

        f = open(fileEvents, "w+")
        f.close()
        
        #create Events-CSV
        eventDF = pandas.DataFrame.from_dict(eventsRange.__dict__["results"])
        eventDF = eventDF[['id', 'name', 'event_url', 'rsvp_limit','yes_rsvp_count', 'time', 'venue','group']]
        eventDF = eventDF.to_json(orient='records')
        # flatten Dateframe
        eventDF = json_normalize(json.loads(eventDF))[['id', 'name', 'event_url', 'rsvp_limit','yes_rsvp_count', 'time','venue.city','group.id', 'group.urlname']].rename(columns={'venue.city': 'city','group.id': 'group_id', 'group.urlname': 'group_urlname'}) 

        eventDF.to_csv(fileEvents, mode='a', encoding='utf-8')

        for index2, row2 in eventDF.iterrows():
            eventId = row2['id']
            groups += row2['group_urlname']+","

            #gather RSVP-data
            rsvp_info = client.GetRsvps({'event_id': eventId})
            rsvpDF = pandas.DataFrame.from_dict(rsvp_info.__dict__["results"])
            #create RSVP-CSV
            rsvpDF.loc[:, ('member')]
            rsvpDF = rsvpDF.to_json(orient='records')
            # flatten Dataframe
            rsvpDF = json_normalize(json.loads(rsvpDF))[['member.name', 'member.member_id']].rename(columns={'member.name': 'name', 'member.member_id': 'member_id'}) 
            rsvpDF['event_id'] = eventId
            rsvpDF.to_csv(fileRSVPs, mode='a', encoding='utf-8')

        #remove last comma
        groups = groups[:-1]
        #gather Group-data
        groupRange = client.GetGroups({'group_urlname' : groups})
        groupDF = pandas.DataFrame.from_dict(groupRange.__dict__["results"])
        #create Group-CSV
        groupDF = groupDF[['id', 'name', 'members', 'link']]                                                             
        groupDF.to_csv(fileGroups, encoding='utf-8')
        

# 3. The events are the basis for the data collection. Via an API, data about the events are fetched in the desired range.
Depending on the desired analysis, the corresponding line is used. The other lines are commented out. 

In [28]:
eventsRangeZH = client.GetOpenEvents(category=34, city="Zürich", country='CH')
# info: 28264920 -> code4fun ---- 31154587 --> swiss data circle
eventsOracle = client.GetEvents({'status': 'past', 'group_id' : '28264920,31154587'})

#eventsRangeBS = client.GetOpenEvents(category=34, city="Basel", country='CH')
#eventsRangeLndn = client.GetOpenEvents(category=34, city="London", country='GB')
#eventsRangeRHF = client.GetOpenEvents(category=34, city="Rheinfelden", country='CH')

#topEvents = client.GetOpenEvents({'category': 34, 'country' : 'CH', 'time' : '1m,12m', 'status' : 'upcoming', 'trending' : 'desc=true'})


29/30 (10 seconds remaining)
28/30 (9 seconds remaining)


# 4. The function for CSV creation is called here using Events-Dataframe. The names of the files to be created are specified in the parameters.
Depending on the desired analysis, the corresponding line is used. The other lines are commented out. 

In [29]:
createCSV("groupsZH.csv", "rsvpZH.csv", "eventsZH.csv", eventsRangeZH)
createCSV("groupsOracle.csv", "rsvpOracle.csv", "eventsOracle.csv", eventsRange)
#createCSV("topEventsGroupsCH.csv", "topEventsRSVPsCH.csv", "topEventsCH.csv", topEvents)

27/30 (3 seconds remaining)
26/30 (2 seconds remaining)
25/30 (2 seconds remaining)
24/30 (2 seconds remaining)
23/30 (1 seconds remaining)
22/30 (1 seconds remaining)
21/30 (1 seconds remaining)
20/30 (1 seconds remaining)
19/30 (0 seconds remaining)
18/30 (0 seconds remaining)
29/30 (10 seconds remaining)
28/30 (10 seconds remaining)
27/30 (10 seconds remaining)
26/30 (9 seconds remaining)
25/30 (9 seconds remaining)
24/30 (9 seconds remaining)
23/30 (9 seconds remaining)
22/30 (8 seconds remaining)
21/30 (8 seconds remaining)
20/30 (8 seconds remaining)
19/30 (7 seconds remaining)
18/30 (7 seconds remaining)
17/30 (7 seconds remaining)
16/30 (6 seconds remaining)
15/30 (6 seconds remaining)
14/30 (6 seconds remaining)
13/30 (6 seconds remaining)
12/30 (6 seconds remaining)
11/30 (5 seconds remaining)
10/30 (5 seconds remaining)
9/30 (5 seconds remaining)
8/30 (4 seconds remaining)
7/30 (4 seconds remaining)
6/30 (4 seconds remaining)
5/30 (3 seconds remaining)
Sleeping for 4 seconds